In [1]:
import numpy as np
from pyscf import gto, scf, fci
from multiprocessing.pool import Pool
from numba import prange, jit, njit
#from numba.openmp import openmp_context as openmp

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy
#                if not(a == b):


In [5]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy

In [6]:
@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [8]:
mol = gto.Mole()
mol.atom = """
    Li    0.    0.    0.
"""
# this basis has 2 functions for Helium
#mol.basis = "6-31g" 
mol.basis = "ccpvdz"
#mol.basis = "sto-6g"
mol.spin=1
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()



(14, 14, 14, 14)
converged SCF energy = -7.43242052759577  <S^2> = 0.75000054  2S+1 = 2.0000005


-7.432420527595772

In [9]:
C = mf.mo_coeff
h = mf.get_hcore()
N = mol.nelec[0]
M = eri.shape[0]
print(C.shape)


P=np.matmul(C[0,:,0:N],C[0,:,0:N].T)




(2, 14, 14)


In [10]:
P = mf.make_rdm1()
print(P.shape)

(2, 14, 14)


In [11]:
P_aa = P[0,:,:]
P_bb = P[1,:,:]

In [12]:
gamma, occu, naturalC = {},{},{}
for i, name in enumerate(['alpha', 'beta']):
    gamma[name] = np.matmul(np.matmul(C[i,:,:].T,np.matmul(np.matmul(S,P[i,:,:]),S)), C[i,:,:])
    occu[name], naturalC[name] = np.linalg.eigh(gamma[name])

In [13]:
# need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
CE = expand_matrix(C[0,:,:])
print(CE.shape, C[1,:,:].shape,N,M,M)
CE[M:,M:]=C[1,:,:]
occu = np.append(occu['alpha'], occu['beta'])
naturalCE = expand_matrix(naturalC['alpha'])
naturalCE[M:,M:] = naturalC['beta']
naturalCTT = np.matmul(CE,naturalCE)

(28, 28) (14, 14) 2 14 14


In [14]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 
print(occu)

[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 1.08652963e-128 1.10953210e-112
 1.09323323e-107 1.37128917e-107 2.12899330e-066 2.29525600e-016
 1.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.39654291e-128 1.08334684e-111 9.69891069e-069
 1.13873144e-065 7.91357354e-051 4.07434703e-018 1.00000000e+000]


In [15]:
#print(naturalCTT)
E_H = ONERDMFT_hartree_energy_parallel(eri, naturalCTT, occu)
E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, naturalCTT, occu)
E_x = ONERDMFT_exchange_energy_parallel(eri, naturalCTT, occu)
E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, naturalCTT, occu)
print(f"Based on expanded HF density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}\nUmrigar Exchangecorrelation: {E_xc}\nUmrigar Hartree + Exchangecorrelation: {E_U + E_xc}")

Based on expanded HF density matrix:
Hartree Energy: 4.062317329440047
Exchange Energy: -1.7812507967560527
Hartree + Exchange 2.2810665326839943
Umrigar Hartree: 2.2950598939116427
Umrigar Exchangecorrelation: -0.013993361746237876
Umrigar Hartree + Exchangecorrelation: 2.2810665321654047


In [16]:
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIgamma = FCIgamma/2.0
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)
FCInaturalCTT = np.matmul(C,FCInaturalC)
FCInaturalCTTE = expand_matrix(FCInaturalCTT)
FCIoccuE = np.append(FCIoccu,FCIoccu)
#print(FCInaturalCTTE)

AssertionError: 1274 1 2

In [ ]:
print(FCIoccu)

In [ ]:
print(f"E_FCI {e} E_HF {mf.e_tot} E_c {e-mf.e_tot}")

In [ ]:
E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + E_xc) - mf.e_tot
print(f"Based on expanded FCI density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}\nUmrigar Exchange-correlation: {E_xc}\nUmrigar Hartree + Exchange-correlation: {E_U + E_xc}")
print(f"Umrigar corelation energy {E_c}")

In [ ]:
h = mf.get_hcore()
2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T))))+E_U+E_xc

In [ ]:
print(np.trace(np.matmul(h,P)))
print(np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))))

In [ ]:

E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + E_xc) - mf.e_tot
print(f"Based on expanded FCI density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Mueller Hartree: {E_U}\nMueller Exchange-correlation: {E_xc}\nMueller Hartree + Exchange-correlation: {E_U + E_xc}")
print(f"Mueller corelation energy {E_c}")

In [ ]:
2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T))))+E_U+E_xc